# Scraping for Question 3

Scraping necessary data to look at the most promising youth players and trying to predict what ratings they would achieve in future events.

Like previous research questions, we will be scraping the data from HLTV.org. Firstly we will be collecting career data of all the players stored on the website.

In [1]:
# Import libraries
import pandas as pd
import urllib
import urllib.request
import time
import re
from loguru import logger
from bs4 import BeautifulSoup

Having the page of all the players ("https://www.hltv.org/stats/players"), I used BeatifulSoup to parse the HTML and a list of all the players pages is created by combining the scraped link and the baseURL. 

In [2]:
playersURL = {"https://www.hltv.org/stats/players"}

baseURL = "https://www.hltv.org"

#dfURL = pd.DataFrame(columns=["URLs"])
URL_List = list()
for page in playersURL:
    
    # Logger message
    logger.info('Scraping new player page: %s.' % page)
    
    # Retrieve HTML
    req = urllib.request.Request(page, headers={"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3"})
    playerList = urllib.request.urlopen(req).read()
    
    # Sleep to prevent overflowing website
    time.sleep(2)
    
    # Parse HTML
    soup = BeautifulSoup(playerList, "html.parser")
    
    for item in soup.find_all(attrs={"class": "playerCol"}):
        for link in item.find_all("a"):
            URL_List.append(baseURL + link.get("href"))

print(URL_List)

2019-05-10 20:25:23.667 | INFO     | __main__:<module>:10 - Scraping new player page: https://www.hltv.org/stats/players.


['https://www.hltv.org/stats/players/11893/ZywOo', 'https://www.hltv.org/stats/players/7998/s1mple', 'https://www.hltv.org/stats/players/7938/XANTARES', 'https://www.hltv.org/stats/players/7131/BnTeT', 'https://www.hltv.org/stats/players/9216/coldzera', 'https://www.hltv.org/stats/players/9102/Sico', 'https://www.hltv.org/stats/players/15631/KSCERATO', 'https://www.hltv.org/stats/players/8950/Kaze', 'https://www.hltv.org/stats/players/10774/Freeman', 'https://www.hltv.org/stats/players/3741/NiKo', 'https://www.hltv.org/stats/players/8711/Sonic', 'https://www.hltv.org/stats/players/9960/frozen', 'https://www.hltv.org/stats/players/11271/smooya', 'https://www.hltv.org/stats/players/9172/XigN', 'https://www.hltv.org/stats/players/12553/yuurih', 'https://www.hltv.org/stats/players/7592/device', 'https://www.hltv.org/stats/players/9626/XELLOW', 'https://www.hltv.org/stats/players/5736/kNgV-', 'https://www.hltv.org/stats/players/8714/cosmeeeN', 'https://www.hltv.org/stats/players/12591/sterl

Having collected a URL list of all the players pages, I downloaded an example page to determine where the nessesary data is being stored. . HLTV is CloudFlare protected, a header parameter is added to the urllib.request to bypass the 403 error returned. A sleep is also used, as too many requests too quickly can result in an IP ban.

After deciding on what data needed to be scraped, I created a dataframe to store all the data call dfPlayerStats.

In [3]:
# Player statistics
dfPlayerStats = pd.DataFrame(columns=["Name", "Age", "Nationality", "Team", "Rating", "Total Kills", "Headshot %", 
                                      "Total deaths", "K/D Ratio", "Damage / Round", "Kills / Round", "Maps played"])

Now that we have a list of the correct URLs, we can scrape each page in a similar way to above, and fill the dataframe with the data.

In [4]:
for URL in URL_List:
    # Retrieve HTML
    req = urllib.request.Request(URL, headers={"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3"})
    playerList = urllib.request.urlopen(req).read()
    
    # Logger message
    logger.info('Scraping new player: %s' % URL)
    
    # Sleep to prevent overflowing website
    time.sleep(2)
    
    # Parse HTML
    soup = BeautifulSoup(playerList, "html.parser")
    
    #Name
    name_box = soup.find(attrs={"class": "statsPlayerName"})
    name = name_box.text.strip()
    
    #Age
    facts_box = soup.find_all(attrs={"class": "large-strong"})
    age = facts_box[1].text.strip()
    nat = facts_box[2].text.strip()
    team = facts_box[3].text.strip()
    
    #Rating
    rating_box = soup.find(attrs={"class": "strong"})
    rating = rating_box.text.strip()
    
    #Stats
    
    stats_box = soup.find_all(attrs={"class": "stats-row"})
    kills = stats_box[0].text.strip()
    headshots = stats_box[1].text.strip()
    deaths = stats_box[2].text.strip()
    kd = stats_box[3].text.strip()
    adr = stats_box[4].text.strip()
    kpr = stats_box[8].text.strip()
    mapsplayed = stats_box[6].text.strip()
    
    dfPlayerStats = dfPlayerStats.append({"Name": name, "Age": age, "Nationality": nat, "Team": team, 
                                          "Rating": rating, "Total Kills": kills, "Headshot %": headshots, 
                                          "Total deaths": deaths, "K/D Ratio": kd, "Damage / Round": adr, "Kills / Round": kpr, 
                                          "Maps played": mapsplayed}, ignore_index=True)
    

2019-05-10 20:25:30.413 | INFO     | __main__:<module>:7 - Scraping new player: https://www.hltv.org/stats/players/11893/ZywOo


2019-05-10 20:25:33.136 | INFO     | __main__:<module>:7 - Scraping new player: https://www.hltv.org/stats/players/7998/s1mple


2019-05-10 20:25:36.556 | INFO     | __main__:<module>:7 - Scraping new player: https://www.hltv.org/stats/players/7938/XANTARES


2019-05-10 20:25:39.536 | INFO     | __main__:<module>:7 - Scraping new player: https://www.hltv.org/stats/players/7131/BnTeT


2019-05-10 20:25:42.915 | INFO     | __main__:<module>:7 - Scraping new player: https://www.hltv.org/stats/players/9216/coldzera


...

2019-05-10 20:58:47.251 | INFO     | __main__:<module>:7 - Scraping new player: https://www.hltv.org/stats/players/7217/SolEk


2019-05-10 20:58:50.128 | INFO     | __main__:<module>:7 - Scraping new player: https://www.hltv.org/stats/players/3545/haste


2019-05-10 20:58:53.473 | INFO     | __main__:<module>:7 - Scraping new player: https://www.hltv.org/stats/players/472/B1ad3


2019-05-10 20:58:56.310 | INFO     | __main__:<module>:7 - Scraping new player: https://www.hltv.org/stats/players/7415/HUNDEN


2019-05-10 20:58:59.241 | INFO     | __main__:<module>:7 - Scraping new player: https://www.hltv.org/stats/players/6455/OCEAN


Having appened the data into the dataframe, excess words are removed from the dataframe to leave just the correct data to be used for further analysis. Some other cleaning is done, as well as putting 'Rating' as the index

In [5]:
dfPlayerStats['Total Kills'] = dfPlayerStats['Total Kills'].map(lambda x: x.lstrip('Total kills'))
dfPlayerStats['Headshot %'] = dfPlayerStats['Headshot %'].map(lambda x: x.lstrip('Headshot %').rstrip('%'))
dfPlayerStats['Total deaths'] = dfPlayerStats['Total deaths'].map(lambda x: x.lstrip('Total deaths'))
dfPlayerStats['K/D Ratio'] = dfPlayerStats['K/D Ratio'].map(lambda x: x.lstrip('K/D Ratio'))
dfPlayerStats['Damage / Round'] = dfPlayerStats['Damage / Round'].map(lambda x: x.lstrip('Damage / Round'))
dfPlayerStats['Kills / Round'] = dfPlayerStats['Kills / Round'].map(lambda x: x.lstrip('Kills / round'))
dfPlayerStats['Maps played'] = dfPlayerStats['Maps played'].map(lambda x: x.lstrip('Maps played'))
dfPlayerStats.set_index("Rating")
dfPlayerStats.sample(5)

,Name,Age,Nationality,Team,Rating,Total Kills,Headshot %,Total deaths,K/D Ratio,Damage / Round,Kills / Round,Maps played
566,Gustav 'ZINKEN' Björk,21,Sweden,nerdRage,0.99,4954,53.9,5237,0.95,71.0,0.64,290
84,Nemanja 'nexa' Isaković,22,Serbia,Valiance,1.10,12770,50.1,10979,1.16,80.2,0.73,668
119,William 'draken' Sundin,23,Sweden,Ancient,1.08,16232,30.9,14236,1.14,74.8,0.73,854
215,Dmitry 'dimasick' Matvienko,22,Kazakhstan,No team,1.09,7390,47.1,6872,1.08,78.9,0.72,397
442,Aidyn 'KrizzeN' Turlybekov,19,Kazakhstan,No team,1.05,9331,56.5,9195,1.01,74.3,0.66,545


In [6]:
dfPlayerStats.isnull().sum()

Name              0
Age               0
Nationality       0
Team              0
Rating            0
Total Kills       0
Headshot %        0
Total deaths      0
K/D Ratio         0
Damage / Round    0
Kills / Round     0
Maps played       0
dtype: int64

There is an error on the website, so I removed the row that contained the player page with the errors in it.

In [7]:
dfPlayerStats = dfPlayerStats[~dfPlayerStats.Name.str.contains("Robin 'Fifflaren' Johansson")]

The dataframe is finally converted to CSV and saved locally.

In [8]:
PlayerStats = dfPlayerStats.to_csv ("ALL_PLAYER_STATS.csv", index = None, header=True)